<a href="https://colab.research.google.com/github/SanjayBista1010/DeepLearning/blob/main/MNISTfromTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
class CNN(nn.Module):
  def __init__(self, in_channels, num_classes=10):
    super().__init__()

    #first convolution layer: 1 input channel, 8 output channels, 3x3 kernel, stride 1, padding 1
    self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=3, stride = 1, padding = 1)

    #max pooling layer: 2x2 window, stride 2
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    #second convolution layer: 8 input channels, 16 o/p channels, 3x3 kernel, stride 1, padding =1
    self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 3, stride = 1, padding = 1)

    #full connected layer: 16*7*7 input features (after two 2x2 poolings), 10 outputs features (num_classes)
    self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

  def forward(self, x):

    x= F.relu(self.conv1(x)) #apply first concolution and ReLU activation
    x= self.pool(x) #apply max pooling
    x = F.relu(self.conv2(x)) #apply sencond convolution and ReLU activation
    x = self.pool(x) #apply max pooling
    x= x.reshape(x.shape[0], -1) #flatten the tensor
    x = self.fc1(x) #apply fully connected layer
    return x

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10

In [ ]:
train_dataset = datasets.MNIST(root='dataset/', download=True, train=True, transform=transforms.ToTensor())
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle = True)

test_dataset = datasets.MNIST(root='dataset/', download=True, train=False, transform=transforms.ToTensor())
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle = True)


In [ ]:
model = CNN(in_channels =1, num_classes=num_classes).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    print(f"Epoch [{epoch + 1}/{num_epochs}]")
    for batch_index, (data, targets) in enumerate(tqdm(train_loader)):
        #move data and targets to the device (GPU/CPU)
        data = data.to(device)
        targets = targets.to(device)

        #forward pass: compute the model output
        scores = model(data)
        loss = criterion(scores, targets)

        #backward pass: compute the gradients
        optimizer.zero_grad()
        loss.backward()

        #optimization step: update the model parameters
        optimizer.step()

Epoch [1/10]


100%|██████████| 938/938 [00:31<00:00, 29.63it/s]


Epoch [2/10]


100%|██████████| 938/938 [00:29<00:00, 31.57it/s]


Epoch [3/10]


...

100%|██████████| 938/938 [00:28<00:00, 32.38it/s]


Epoch [9/10]


100%|██████████| 938/938 [00:29<00:00, 32.28it/s]


Epoch [10/10]


100%|██████████| 938/938 [00:28<00:00, 32.95it/s]


In [ ]:
def check_accuracy(loader, model):

    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():  #disable gradient calculation
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            #forward pass: compute the model output
            scores = model(x)
            _, predictions = scores.max(1)  #get the index of the max log-probability
            num_correct += (predictions == y).sum()  #count correct predictions
            num_samples += predictions.size(0)  #count total samples

        #calculate accuracy
        accuracy = float(num_correct) / float(num_samples) * 100
        print(f"Got {num_correct}/{num_samples} with accuracy {accuracy:.2f}%")

    model.train()  #set the model back to training mode

#final accuracy check on training and test sets
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 59487/60000 with accuracy 99.15%
Checking accuracy on test data
Got 9861/10000 with accuracy 98.61%
